# Final Project Submission

Please fill out:
* Student name: Hiten Patel 
* Student pace: Full Time Online 
* Scheduled project review date/time: 10/17: 10AM
* Instructor name: James Irving 
* Blog post URL: https://hpatel530.github.io/blog_number_4_by_hiten_patel


# Chicago Car Crash: Predicting the Cause of Traffic Accidents 

## Introduction 

# Obtain 

## Import Packages 

In [78]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, r2_score, recall_score, precision_score, roc_curve, roc_auc_score
from sklearn.metrics import classification_report #, plot_roc_curve , plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler

from sklearn.compose import ColumnTransformer
# from sklearn import set_config
# set_config(display='diagram')

import warnings
warnings.filterwarnings('ignore')

## Import Data

### Data Source 1: crash 

In [2]:
# importing database one and observe 
pd.set_option('display.max_columns', None)
crash = pd.read_csv('https://data.cityofchicago.org/resource/85ca-t3if.csv')
print(crash.shape)
display(crash.head())
crash.info()

(1000, 49)


,crash_record_id,rd_no,crash_date_est_i,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,lane_cnt,alignment,roadway_surface_cond,road_defect,report_type,crash_type,intersection_related_i,private_property_i,hit_and_run_i,damage,date_police_notified,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,photos_taken_i,statements_taken_i,dooring_i,work_zone_i,work_zone_type,workers_present_i,num_units,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,location
0,11f701454825c47eba7689340eccbf1877377ab49703b9...,NaN,NaN,2020-10-08T00:30:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,OTHER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,NaN,N,NaN,$500 OR LESS,2020-10-08T00:30:00.000,IMPROPER BACKING,IMPROPER BACKING,6100,W,FULLERTON AVE,2512,NaN,NaN,NaN,NaN,NaN,NaN,1,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0,5,10,41.923898,-87.778211,POINT (-87.77821097882 41.923897721686)
1,7b5500f750855513626cf80099800e991ac34675fd5089...,NaN,NaN,2020-10-07T22:20:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,NaN,STRAIGHT AND LEVEL,DRY,SHOULDER DEFECT,ON SCENE,NO INJURY / DRIVE AWAY,NaN,NaN,Y,"OVER $1,500",2020-10-07T22:26:00.000,UNABLE TO DETERMINE,NOT APPLICABLE,5056,W,GLADYS AVE,1533,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0,22,4,10,41.875907,-87.751584,POINT (-87.751583949887 41.875907198646)
2,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,NaN,NaN,2020-10-07T22:00:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PEDESTRIAN,ALLEY,NaN,STRAIGHT AND LEVEL,DRY,UNKNOWN,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,NaN,NaN,Y,"OVER $1,500",2020-10-07T22:48:00.000,UNABLE TO DETERMINE,NOT APPLICABLE,5150,W,CHICAGO AVE,1531,NaN,NaN,NaN,NaN,NaN,NaN,2,"REPORTED, NOT EVIDENT",1.0,0.0,0.0,0.0,1.0,1.0,0.0,22,4,10,41.895054,-87.755225,POINT (-87.755225290757 41.895053730813)
3,3de84b4b2407b3274f6b6a554caa05e674e20bfd5bc27f...,NaN,NaN,2020-10-07T21:50:00.000,30,STOP SIGN/FLASHER,NO CONTROLS,CLEAR,DARKNESS,TURNING,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,Y,NaN,NaN,"$501 - $1,500",2020-10-07T21:50:00.000,UNABLE TO DETERMINE,NOT APPLICABLE,3300,S,HALSTED ST,913,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,0.0,21,4,10,41.834808,-87.646349,POINT (-87.646348729083 41.834807976807)
4,fe12e88dc1ee12153f44f94e1af1e9bb7684fb25ef34fc...,NaN,NaN,2020-10-07T21:41:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,NaN,NaN,Y,"OVER $1,500",2020-10-07T21:42:00.000,HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE),NOT APPLICABLE,835,N,AUSTIN BLVD,1511,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0,21,4,10,41.895667,-87.775014,POINT (-87.775014447162 41.895666936337)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   crash_record_id                1000 non-null   object 
 1   rd_no                          0 non-null      float64
 2   crash_date_est_i               69 non-null     object 
 3   crash_date                     1000 non-null   object 
 4   posted_speed_limit             1000 non-null   int64  
 5   traffic_control_device         1000 non-null   object 
 6   device_condition               1000 non-null   object 
 7   weather_condition              1000 non-null   object 
 8   lighting_condition             1000 non-null   object 
 9   first_crash_type               1000 non-null   object 
 10  trafficway_type                1000 non-null   object 
 11  lane_cnt                       0 non-null      float64
 12  alignment                      1000 non-null   ob

### Data Source 2: vehicle 


In [3]:
# importing database two and observe 
vehicle = pd.read_csv('https://data.cityofchicago.org/resource/68nd-jvt3.csv')
print(vehicle.shape)
display(vehicle.head())
vehicle.info()

(1000, 72)


,crash_unit_id,crash_record_id,rd_no,crash_date,unit_no,unit_type,num_passengers,vehicle_id,cmrc_veh_i,make,model,lic_plate_state,vehicle_year,vehicle_defect,vehicle_type,vehicle_use,travel_direction,maneuver,towed_i,fire_i,occupant_cnt,exceed_speed_limit_i,towed_by,towed_to,area_00_i,area_01_i,area_02_i,area_03_i,area_04_i,area_05_i,area_06_i,area_07_i,area_08_i,area_09_i,area_10_i,area_11_i,area_12_i,area_99_i,first_contact_point,cmv_id,usdot_no,ccmc_no,ilcc_no,commercial_src,gvwr,carrier_name,carrier_state,carrier_city,hazmat_placards_i,hazmat_name,un_no,hazmat_present_i,hazmat_report_i,hazmat_report_no,mcs_report_i,mcs_report_no,hazmat_vio_cause_crash_i,mcs_vio_cause_crash_i,idot_permit_no,wide_load_i,trailer1_width,trailer2_width,trailer1_length,trailer2_length,total_vehicle_length,axle_cnt,vehicle_config,cargo_body_type,load_type,hazmat_out_of_service_i,mcs_out_of_service_i,hazmat_class
0,972730,11f701454825c47eba7689340eccbf1877377ab49703b9...,NaN,2020-10-08T00:30:00.000,1,DRIVER,1.0,921874.0,NaN,FORD,EXPLORER,IL,2017.0,NONE,PASSENGER,POLICE,N,BACKING,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,N,NaN,NaN,NaN,NaN,NaN,NaN,REAR-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,972705,7b5500f750855513626cf80099800e991ac34675fd5089...,NaN,2020-10-07T22:20:00.000,1,DRIVER,NaN,921851.0,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),IL,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,W,UNKNOWN/NA,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRONT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,972706,7b5500f750855513626cf80099800e991ac34675fd5089...,NaN,2020-10-07T22:20:00.000,2,PARKED,NaN,921853.0,NaN,HONDA,ACCORD,IL,1995.0,NONE,PASSENGER,NOT IN USE,W,PARKED,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,SIDE-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,972718,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,NaN,2020-10-07T22:00:00.000,1,DRIVER,NaN,921864.0,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),IL,NaN,UNKNOWN,PASSENGER,UNKNOWN/NA,E,STRAIGHT AHEAD,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,972719,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,NaN,2020-10-07T22:00:00.000,2,PEDESTRIAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 72 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   crash_unit_id             1000 non-null   int64  
 1   crash_record_id           1000 non-null   object 
 2   rd_no                     0 non-null      float64
 3   crash_date                1000 non-null   object 
 4   unit_no                   1000 non-null   int64  
 5   unit_type                 1000 non-null   object 
 6   num_passengers            137 non-null    float64
 7   vehicle_id                975 non-null    float64
 8   cmrc_veh_i                18 non-null     object 
 9   make                      975 non-null    object 
 10  model                     975 non-null    object 
 11  lic_plate_state           876 non-null    object 
 12  vehicle_year              799 non-null    float64
 13  vehicle_defect            975 non-null    object 
 14  vehicle_t

### Data Source 3: person 

In [4]:
# importing database three and observe 
person = pd.read_csv('https://data.cityofchicago.org/resource/u6pd-qa9d.csv')
print(person.shape)
display(person.head())
person.info()

(1000, 30)


,person_id,person_type,crash_record_id,rd_no,vehicle_id,crash_date,seat_no,city,state,zipcode,sex,age,drivers_license_state,drivers_license_class,safety_equipment,airbag_deployed,ejection,injury_classification,hospital,ems_agency,ems_run_no,driver_action,driver_vision,physical_condition,pedpedal_action,pedpedal_visibility,pedpedal_location,bac_result,bac_result_value,cell_phone_use
0,O972730,DRIVER,11f701454825c47eba7689340eccbf1877377ab49703b9...,NaN,921874.0,2020-10-08T00:30:00.000,NaN,CHICAGO,IL,60639,M,37.0,IL,D,SAFETY BELT USED,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,IMPROPER BACKING,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,P221360,PASSENGER,11f701454825c47eba7689340eccbf1877377ab49703b9...,NaN,921874.0,2020-10-08T00:30:00.000,2.0,CHICAGO,IL,60639,M,28.0,NaN,NaN,SAFETY BELT USED,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O972705,DRIVER,7b5500f750855513626cf80099800e991ac34675fd5089...,NaN,921851.0,2020-10-07T22:20:00.000,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O972718,DRIVER,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,NaN,921864.0,2020-10-07T22:00:00.000,NaN,NaN,NaN,NaN,M,NaN,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O972719,PEDESTRIAN,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,NaN,NaN,2020-10-07T22:00:00.000,NaN,CHICAGO,IL,60644,F,20.0,NaN,NaN,NaN,NaN,NaN,"REPORTED, NOT EVIDENT",STROGER,CFD AMB 15,NaN,NONE,UNKNOWN,UNKNOWN,UNKNOWN/NA,NO CONTRASTING CLOTHING,UNKNOWN/NA,TEST NOT OFFERED,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   person_id              1000 non-null   object 
 1   person_type            1000 non-null   object 
 2   crash_record_id        1000 non-null   object 
 3   rd_no                  0 non-null      float64
 4   vehicle_id             975 non-null    float64
 5   crash_date             1000 non-null   object 
 6   seat_no                175 non-null    float64
 7   city                   725 non-null    object 
 8   state                  731 non-null    object 
 9   zipcode                659 non-null    object 
 10  sex                    990 non-null    object 
 11  age                    704 non-null    float64
 12  drivers_license_state  620 non-null    object 
 13  drivers_license_class  521 non-null    object 
 14  safety_equipment       997 non-null    object 
 15  airba

## Functions 

# Scrub 

## Data Source 1: Remove Irrelevant Columns 

In [5]:
# dropping following columns due to irrelevance in predicting the cause of car accidents 
# irrelevent columns were dropped due to column description
drop = ['rd_no', 'crash_date_est_i', 'report_type', 'crash_type', 'damage', 'date_police_notified', 
        'photos_taken_i', 'statements_taken_i', 'injuries_fatal', 'injuries_incapacitating', 'most_severe_injury',
        'injuries_total', 'injuries_non_incapacitating','injuries_reported_not_evident', 'injuries_no_indication', 'injuries_unknown']
crash = crash.drop(columns = drop)
print(crash.shape)
crash.head()

(1000, 33)


,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,lane_cnt,alignment,roadway_surface_cond,road_defect,intersection_related_i,private_property_i,hit_and_run_i,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,dooring_i,work_zone_i,work_zone_type,workers_present_i,num_units,crash_hour,crash_day_of_week,crash_month,latitude,longitude,location
0,11f701454825c47eba7689340eccbf1877377ab49703b9...,2020-10-08T00:30:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,OTHER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,N,NaN,IMPROPER BACKING,IMPROPER BACKING,6100,W,FULLERTON AVE,2512,NaN,NaN,NaN,NaN,1,0,5,10,41.923898,-87.778211,POINT (-87.77821097882 41.923897721686)
1,7b5500f750855513626cf80099800e991ac34675fd5089...,2020-10-07T22:20:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,NaN,STRAIGHT AND LEVEL,DRY,SHOULDER DEFECT,NaN,NaN,Y,UNABLE TO DETERMINE,NOT APPLICABLE,5056,W,GLADYS AVE,1533,NaN,NaN,NaN,NaN,2,22,4,10,41.875907,-87.751584,POINT (-87.751583949887 41.875907198646)
2,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,2020-10-07T22:00:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PEDESTRIAN,ALLEY,NaN,STRAIGHT AND LEVEL,DRY,UNKNOWN,NaN,NaN,Y,UNABLE TO DETERMINE,NOT APPLICABLE,5150,W,CHICAGO AVE,1531,NaN,NaN,NaN,NaN,2,22,4,10,41.895054,-87.755225,POINT (-87.755225290757 41.895053730813)
3,3de84b4b2407b3274f6b6a554caa05e674e20bfd5bc27f...,2020-10-07T21:50:00.000,30,STOP SIGN/FLASHER,NO CONTROLS,CLEAR,DARKNESS,TURNING,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,Y,NaN,NaN,UNABLE TO DETERMINE,NOT APPLICABLE,3300,S,HALSTED ST,913,NaN,NaN,NaN,NaN,2,21,4,10,41.834808,-87.646349,POINT (-87.646348729083 41.834807976807)
4,fe12e88dc1ee12153f44f94e1af1e9bb7684fb25ef34fc...,2020-10-07T21:41:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NaN,Y,HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE),NOT APPLICABLE,835,N,AUSTIN BLVD,1511,NaN,NaN,NaN,NaN,2,21,4,10,41.895667,-87.775014,POINT (-87.775014447162 41.895666936337)


## Data Source 2: Remove Irrelevant Columns 

In [6]:
#dropped duplicate columns found in the first data source and irrevelant columns
#irrelevent columns were dropped due to column description
drop = ['rd_no', 'crash_date', 'towed_i', 'fire_i', 'towed_by', 'towed_to', 'area_00_i', 
      'area_01_i', 'area_02_i', 'area_03_i', 'area_04_i', 'area_05_i', 'area_06_i', 'area_07_i',
      'area_08_i', 'area_09_i', 'area_10_i', 'area_11_i', 'area_12_i', 'area_99_i']
vehicle = vehicle.drop(columns=drop)
print(vehicle.shape)
vehicle.head()

(1000, 52)


,crash_unit_id,crash_record_id,unit_no,unit_type,num_passengers,vehicle_id,cmrc_veh_i,make,model,lic_plate_state,vehicle_year,vehicle_defect,vehicle_type,vehicle_use,travel_direction,maneuver,occupant_cnt,exceed_speed_limit_i,first_contact_point,cmv_id,usdot_no,ccmc_no,ilcc_no,commercial_src,gvwr,carrier_name,carrier_state,carrier_city,hazmat_placards_i,hazmat_name,un_no,hazmat_present_i,hazmat_report_i,hazmat_report_no,mcs_report_i,mcs_report_no,hazmat_vio_cause_crash_i,mcs_vio_cause_crash_i,idot_permit_no,wide_load_i,trailer1_width,trailer2_width,trailer1_length,trailer2_length,total_vehicle_length,axle_cnt,vehicle_config,cargo_body_type,load_type,hazmat_out_of_service_i,mcs_out_of_service_i,hazmat_class
0,972730,11f701454825c47eba7689340eccbf1877377ab49703b9...,1,DRIVER,1.0,921874.0,NaN,FORD,EXPLORER,IL,2017.0,NONE,PASSENGER,POLICE,N,BACKING,2.0,NaN,REAR-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,972705,7b5500f750855513626cf80099800e991ac34675fd5089...,1,DRIVER,NaN,921851.0,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),IL,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,W,UNKNOWN/NA,1.0,NaN,FRONT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,972706,7b5500f750855513626cf80099800e991ac34675fd5089...,2,PARKED,NaN,921853.0,NaN,HONDA,ACCORD,IL,1995.0,NONE,PASSENGER,NOT IN USE,W,PARKED,0.0,NaN,SIDE-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,972718,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,1,DRIVER,NaN,921864.0,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),IL,NaN,UNKNOWN,PASSENGER,UNKNOWN/NA,E,STRAIGHT AHEAD,1.0,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,972719,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,2,PEDESTRIAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Source 3: Remove Irrelevant Columns

In [7]:
print(person.shape)
drop = ['rd_no', 'crash_date', 'seat_no', 'safety_equipment', 'airbag_deployed', 'ejection',
       'injury_classification', 'hospital', 'ems_agency', 'ems_run_no']
person = person.drop(columns=drop)
print(person.shape)
person.head()

(1000, 30)
(1000, 20)


,person_id,person_type,crash_record_id,vehicle_id,city,state,zipcode,sex,age,drivers_license_state,drivers_license_class,driver_action,driver_vision,physical_condition,pedpedal_action,pedpedal_visibility,pedpedal_location,bac_result,bac_result_value,cell_phone_use
0,O972730,DRIVER,11f701454825c47eba7689340eccbf1877377ab49703b9...,921874.0,CHICAGO,IL,60639,M,37.0,IL,D,IMPROPER BACKING,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,P221360,PASSENGER,11f701454825c47eba7689340eccbf1877377ab49703b9...,921874.0,CHICAGO,IL,60639,M,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O972705,DRIVER,7b5500f750855513626cf80099800e991ac34675fd5089...,921851.0,NaN,NaN,NaN,X,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O972718,DRIVER,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,921864.0,NaN,NaN,NaN,M,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O972719,PEDESTRIAN,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,NaN,CHICAGO,IL,60644,F,20.0,NaN,NaN,NONE,UNKNOWN,UNKNOWN,UNKNOWN/NA,NO CONTRASTING CLOTHING,UNKNOWN/NA,TEST NOT OFFERED,NaN,NaN


## Merging Data = df 

In [8]:
#merging all three data sources into 1, observing shape preview data 
merged = pd.merge(left=crash, right = vehicle, left_on='crash_record_id', right_on="crash_record_id")
df = pd.merge(left=merged, right=person, left_on = 'vehicle_id', right_on='vehicle_id')
print(df.shape)
df.head()

(1600, 103)


,crash_record_id_x,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,lane_cnt,alignment,roadway_surface_cond,road_defect,intersection_related_i,private_property_i,hit_and_run_i,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,dooring_i,work_zone_i,work_zone_type,workers_present_i,num_units,crash_hour,crash_day_of_week,crash_month,latitude,longitude,location,crash_unit_id,unit_no,unit_type,num_passengers,vehicle_id,cmrc_veh_i,make,model,lic_plate_state,vehicle_year,vehicle_defect,vehicle_type,vehicle_use,travel_direction,maneuver,occupant_cnt,exceed_speed_limit_i,first_contact_point,cmv_id,usdot_no,ccmc_no,ilcc_no,commercial_src,gvwr,carrier_name,carrier_state,carrier_city,hazmat_placards_i,hazmat_name,un_no,hazmat_present_i,hazmat_report_i,hazmat_report_no,mcs_report_i,mcs_report_no,hazmat_vio_cause_crash_i,mcs_vio_cause_crash_i,idot_permit_no,wide_load_i,trailer1_width,trailer2_width,trailer1_length,trailer2_length,total_vehicle_length,axle_cnt,vehicle_config,cargo_body_type,load_type,hazmat_out_of_service_i,mcs_out_of_service_i,hazmat_class,person_id,person_type,crash_record_id_y,city,state,zipcode,sex,age,drivers_license_state,drivers_license_class,driver_action,driver_vision,physical_condition,pedpedal_action,pedpedal_visibility,pedpedal_location,bac_result,bac_result_value,cell_phone_use
0,11f701454825c47eba7689340eccbf1877377ab49703b9...,2020-10-08T00:30:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,OTHER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,N,NaN,IMPROPER BACKING,IMPROPER BACKING,6100,W,FULLERTON AVE,2512,NaN,NaN,NaN,NaN,1,0,5,10,41.923898,-87.778211,POINT (-87.77821097882 41.923897721686),972730,1,DRIVER,1.0,921874.0,NaN,FORD,EXPLORER,IL,2017.0,NONE,PASSENGER,POLICE,N,BACKING,2.0,NaN,REAR-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O972730,DRIVER,11f701454825c47eba7689340eccbf1877377ab49703b9...,CHICAGO,IL,60639,M,37.0,IL,D,IMPROPER BACKING,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,11f701454825c47eba7689340eccbf1877377ab49703b9...,2020-10-08T00:30:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,OTHER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,N,NaN,IMPROPER BACKING,IMPROPER BACKING,6100,W,FULLERTON AVE,2512,NaN,NaN,NaN,NaN,1,0,5,10,41.923898,-87.778211,POINT (-87.77821097882 41.923897721686),972730,1,DRIVER,1.0,921874.0,NaN,FORD,EXPLORER,IL,2017.0,NONE,PASSENGER,POLICE,N,BACKING,2.0,NaN,REAR-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P221360,PASSENGER,11f701454825c47eba7689340eccbf1877377ab49703b9...,CHICAGO,IL,60639,M,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7b5500f750855513626cf80099800e991ac34675fd5089...,2020-10-07T22:20:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,NaN,STRAIGHT AND LEVEL,DRY,SHOULDER DEFECT,NaN,NaN,Y,UNABLE TO DETERMINE,NOT APPLICABLE,5056,W,GLADYS AVE,1533,NaN,NaN,NaN,NaN,2,22,4,10,41.875907,-87.751584,POINT (-87.751583949887 41.875907198646),972705,1,DRIVER,NaN,921851.0,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),IL,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,W,UNKNOWN/NA,1.0,NaN,FRONT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O972705,DRIVER,7b5500f750855513626cf80099800e991ac34675fd5089...,NaN,NaN,NaN,X,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,2020-10-07T22:00:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PEDESTRIAN,ALLEY,NaN,STRAIGHT AND LEVEL,DRY,UNKNOWN,NaN,NaN,Y,UNABLE TO DETERMINE,NOT APPLICABLE,5150,W,CHICAGO AVE,1531,NaN,NaN,NaN,NaN,2,22,4,10,41.895054,-87.75522

## Calculating Nulls/Column 

In [9]:
# calculating nulls
nulls = df.isna().sum()
null_percent = nulls[nulls>0] / len(df)
null_percent.to_frame('% Null').style.background_gradient(cmap='Reds')

,% Null
lane_cnt,1.000000
intersection_related_i,0.616250
private_property_i,0.966875
hit_and_run_i,0.635625
dooring_i,0.967500
work_zone_i,0.996875
work_zone_type,0.998125
workers_present_i,0.998125
latitude,0.001875
longitude,0.001875


In [10]:
# extracting columns with excesssive nulls 
Index_label = null_percent[null_percent>.95].index.tolist()
Index_label

['lane_cnt',
 'private_property_i',
 'dooring_i',
 'work_zone_i',
 'work_zone_type',
 'workers_present_i',
 'cmrc_veh_i',
 'exceed_speed_limit_i',
 'cmv_id',
 'usdot_no',
 'ccmc_no',
 'ilcc_no',
 'commercial_src',
 'gvwr',
 'carrier_name',
 'carrier_state',
 'carrier_city',
 'hazmat_placards_i',
 'hazmat_name',
 'un_no',
 'hazmat_present_i',
 'hazmat_report_i',
 'hazmat_report_no',
 'mcs_report_i',
 'mcs_report_no',
 'hazmat_vio_cause_crash_i',
 'mcs_vio_cause_crash_i',
 'idot_permit_no',
 'wide_load_i',
 'trailer1_width',
 'trailer2_width',
 'trailer1_length',
 'trailer2_length',
 'total_vehicle_length',
 'axle_cnt',
 'vehicle_config',
 'cargo_body_type',
 'load_type',
 'hazmat_out_of_service_i',
 'mcs_out_of_service_i',
 'hazmat_class',
 'bac_result_value',
 'cell_phone_use']

## Dropping Columns w/ Excessive Nulls

In [11]:
df = df.drop(columns = Index_label)
print(df.shape)
display(df.head())
df.info()

(1600, 60)


,crash_record_id_x,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,intersection_related_i,hit_and_run_i,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,num_units,crash_hour,crash_day_of_week,crash_month,latitude,longitude,location,crash_unit_id,unit_no,unit_type,num_passengers,vehicle_id,make,model,lic_plate_state,vehicle_year,vehicle_defect,vehicle_type,vehicle_use,travel_direction,maneuver,occupant_cnt,first_contact_point,person_id,person_type,crash_record_id_y,city,state,zipcode,sex,age,drivers_license_state,drivers_license_class,driver_action,driver_vision,physical_condition,pedpedal_action,pedpedal_visibility,pedpedal_location,bac_result
0,11f701454825c47eba7689340eccbf1877377ab49703b9...,2020-10-08T00:30:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NaN,IMPROPER BACKING,IMPROPER BACKING,6100,W,FULLERTON AVE,2512,1,0,5,10,41.923898,-87.778211,POINT (-87.77821097882 41.923897721686),972730,1,DRIVER,1.0,921874.0,FORD,EXPLORER,IL,2017.0,NONE,PASSENGER,POLICE,N,BACKING,2.0,REAR-LEFT,O972730,DRIVER,11f701454825c47eba7689340eccbf1877377ab49703b9...,CHICAGO,IL,60639,M,37.0,IL,D,IMPROPER BACKING,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED
1,11f701454825c47eba7689340eccbf1877377ab49703b9...,2020-10-08T00:30:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NaN,IMPROPER BACKING,IMPROPER BACKING,6100,W,FULLERTON AVE,2512,1,0,5,10,41.923898,-87.778211,POINT (-87.77821097882 41.923897721686),972730,1,DRIVER,1.0,921874.0,FORD,EXPLORER,IL,2017.0,NONE,PASSENGER,POLICE,N,BACKING,2.0,REAR-LEFT,P221360,PASSENGER,11f701454825c47eba7689340eccbf1877377ab49703b9...,CHICAGO,IL,60639,M,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7b5500f750855513626cf80099800e991ac34675fd5089...,2020-10-07T22:20:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,SHOULDER DEFECT,NaN,Y,UNABLE TO DETERMINE,NOT APPLICABLE,5056,W,GLADYS AVE,1533,2,22,4,10,41.875907,-87.751584,POINT (-87.751583949887 41.875907198646),972705,1,DRIVER,NaN,921851.0,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),IL,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,W,UNKNOWN/NA,1.0,FRONT,O972705,DRIVER,7b5500f750855513626cf80099800e991ac34675fd5089...,NaN,NaN,NaN,X,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED
3,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,2020-10-07T22:00:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PEDESTRIAN,ALLEY,STRAIGHT AND LEVEL,DRY,UNKNOWN,NaN,Y,UNABLE TO DETERMINE,NOT APPLICABLE,5150,W,CHICAGO AVE,1531,2,22,4,10,41.895054,-87.755225,POINT (-87.755225290757 41.895053730813),972718,1,DRIVER,NaN,921864.0,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),IL,NaN,UNKNOWN,PASSENGER,UNKNOWN/NA,E,STRAIGHT AHEAD,1.0,UNKNOWN,O972718,DRIVER,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,NaN,NaN,NaN,M,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED
4,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,2020-10-07T22:00:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PEDESTRIAN,ALLEY,STRAIGHT AND LEVEL,DRY,UNKNOWN,NaN,Y,UNABLE TO DETERMINE,NOT APPLICABLE,5150,W,CHICAGO AVE,1531,2,22,4,10,41.895054,-87.755225,POINT (-87.755225290757 41.895053730813),972719,2,PEDESTRIAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O972719,PEDESTRIAN,c3890ea745b8920c73f3c0b67537fe8928e789d997eed5...,CHICAGO,IL,60644,F,20.0,NaN,NaN,NONE,UNKNOWN,UNKNOWN,UNKNOWN/NA,NO CONTRASTING CLOTHING,UNKNOWN/NA,TEST NOT OFFERED


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 0 to 1599
Data columns (total 60 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   crash_record_id_x        1600 non-null   object 
 1   crash_date               1600 non-null   object 
 2   posted_speed_limit       1600 non-null   int64  
 3   traffic_control_device   1600 non-null   object 
 4   device_condition         1600 non-null   object 
 5   weather_condition        1600 non-null   object 
 6   lighting_condition       1600 non-null   object 
 7   first_crash_type         1600 non-null   object 
 8   trafficway_type          1600 non-null   object 
 9   alignment                1600 non-null   object 
 10  roadway_surface_cond     1600 non-null   object 
 11  road_defect              1600 non-null   object 
 12  intersection_related_i   614 non-null    object 
 13  hit_and_run_i            583 non-null    object 
 14  prim_contributory_cause 

## Dropping Redundant Columns

In [12]:
#dropping redundant/irrelevant columns 

drop = ['crash_record_id_x','crash_unit_id', 'vehicle_id', 'person_id', "crash_record_id_y", 'street_no', 'street_direction',
       'street_name', 'location', 'zipcode']
df = df.drop(columns=drop)
print(df.shape)
df.head()

(1600, 50)


,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,intersection_related_i,hit_and_run_i,prim_contributory_cause,sec_contributory_cause,beat_of_occurrence,num_units,crash_hour,crash_day_of_week,crash_month,latitude,longitude,unit_no,unit_type,num_passengers,make,model,lic_plate_state,vehicle_year,vehicle_defect,vehicle_type,vehicle_use,travel_direction,maneuver,occupant_cnt,first_contact_point,person_type,city,state,sex,age,drivers_license_state,drivers_license_class,driver_action,driver_vision,physical_condition,pedpedal_action,pedpedal_visibility,pedpedal_location,bac_result
0,2020-10-08T00:30:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NaN,IMPROPER BACKING,IMPROPER BACKING,2512,1,0,5,10,41.923898,-87.778211,1,DRIVER,1.0,FORD,EXPLORER,IL,2017.0,NONE,PASSENGER,POLICE,N,BACKING,2.0,REAR-LEFT,DRIVER,CHICAGO,IL,M,37.0,IL,D,IMPROPER BACKING,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED
1,2020-10-08T00:30:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NaN,IMPROPER BACKING,IMPROPER BACKING,2512,1,0,5,10,41.923898,-87.778211,1,DRIVER,1.0,FORD,EXPLORER,IL,2017.0,NONE,PASSENGER,POLICE,N,BACKING,2.0,REAR-LEFT,PASSENGER,CHICAGO,IL,M,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-10-07T22:20:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,SHOULDER DEFECT,NaN,Y,UNABLE TO DETERMINE,NOT APPLICABLE,1533,2,22,4,10,41.875907,-87.751584,1,DRIVER,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),IL,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,W,UNKNOWN/NA,1.0,FRONT,DRIVER,NaN,NaN,X,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED
3,2020-10-07T22:00:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PEDESTRIAN,ALLEY,STRAIGHT AND LEVEL,DRY,UNKNOWN,NaN,Y,UNABLE TO DETERMINE,NOT APPLICABLE,1531,2,22,4,10,41.895054,-87.755225,1,DRIVER,NaN,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),IL,NaN,UNKNOWN,PASSENGER,UNKNOWN/NA,E,STRAIGHT AHEAD,1.0,UNKNOWN,DRIVER,NaN,NaN,M,NaN,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED
4,2020-10-07T22:00:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PEDESTRIAN,ALLEY,STRAIGHT AND LEVEL,DRY,UNKNOWN,NaN,Y,UNABLE TO DETERMINE,NOT APPLICABLE,1531,2,22,4,10,41.895054,-87.755225,2,PEDESTRIAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PEDESTRIAN,CHICAGO,IL,F,20.0,NaN,NaN,NONE,UNKNOWN,UNKNOWN,UNKNOWN/NA,NO CONTRASTING CLOTHING,UNKNOWN/NA,TEST NOT OFFERED


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 0 to 1599
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   crash_date               1600 non-null   object 
 1   posted_speed_limit       1600 non-null   int64  
 2   traffic_control_device   1600 non-null   object 
 3   device_condition         1600 non-null   object 
 4   weather_condition        1600 non-null   object 
 5   lighting_condition       1600 non-null   object 
 6   first_crash_type         1600 non-null   object 
 7   trafficway_type          1600 non-null   object 
 8   alignment                1600 non-null   object 
 9   roadway_surface_cond     1600 non-null   object 
 10  road_defect              1600 non-null   object 
 11  intersection_related_i   614 non-null    object 
 12  hit_and_run_i            583 non-null    object 
 13  prim_contributory_cause  1600 non-null   object 
 14  sec_contributory_cause  

In [23]:
df.nunique()

crash_date                 359
posted_speed_limit          10
traffic_control_device       9
device_condition             6
weather_condition            4
lighting_condition           6
first_crash_type            14
trafficway_type             17
alignment                    5
roadway_surface_cond         3
road_defect                  5
intersection_related_i       2
hit_and_run_i                2
prim_contributory_cause     26
sec_contributory_cause      23
beat_of_occurrence         210
num_units                    5
crash_hour                  24
crash_day_of_week            4
crash_month                  1
latitude                   464
longitude                  464
unit_no                      5
unit_type                    5
num_passengers               4
make                        52
model                      216
lic_plate_state             24
vehicle_year                31
vehicle_defect               6
vehicle_type                13
vehicle_use                 14
travel_d

In [38]:
df.hit_and_run_i.value_counts()

Y    544
N     39
Name: hit_and_run_i, dtype: int64

In [39]:
df.intersection_related_i.value_counts()

Y    579
N     35
Name: intersection_related_i, dtype: int64

## Engineering New Columns

Beat_of_occurrence = part of district because it has been divided ### how to connect to map?
lat and long = geocode  

target variable = age (bins, driver)

## Dropping Irrelevant Columns 

## Final Scrubbed Dataframe

# Explore

# Train-Test-Split


In [55]:
# setting X,y for train-test-split
target = 'sex'
X = df.drop(columns= target)
y = df[target]

# train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=42)

## Preprocessing Numerical 

In [56]:
# retrieving list of numeric columns 
num_cols = df.drop(columns=target).select_dtypes('number').columns.tolist()

In [57]:
# creating a pipeline 
# SimpleImputer will use the median to fill nulls  
# RobustScaler will use the median 

num_transform = Pipeline([('imputer', SimpleImputer(strategy='median')),
                         ('scale', RobustScaler())])

## Preprocessing Categorical 

In [58]:
# retrieving list of categorical columns 
cat_cols = df.drop(columns=target).select_dtypes('object').columns.tolist()

In [59]:
# creating a pipeline
# SimpleImputer will use 
# OneHotEncoder will scale the categorical data to a binary column
cat_transform = Pipeline([('imputer', SimpleImputer(strategy='constant')),
                         ('encoder', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

## ColumnTransformer

In [60]:
# combine both pipelines into one using columntransformer 

preprocessing = ColumnTransformer([('num', num_transform, num_cols), 
                                  ('cat', cat_transform, cat_cols)])

## Class Imbalance (SMOTE)

# Model 

In [75]:
def evaluate_classification(model, X_train, X_test, y_train, y_test, classes = None, 
                            normalize = 'true', cmap='Purples', label = ''):
    
    """Input a model, training data and test data to return sklearn metrics
        - Classification Report for training and test 
        - Confusion Matrix 
        - ROC Curve 
    """
    # retrieve predictions for train and test data 
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # print training classification report 
    header = label + " CLASSIFICATION REPORT TRAINING "
    dashes = "---" * 20
    print(dashes, header, dashes, sep='\n')
    print(classification_report(y_train, y_pred_train, target_names = classes))
    
    # display training figures 
    fig, axes = plt.subplots(figsize=(10,4), ncols=2)
    
    # confusion matrix 
    plot_confusion_matrix(model, X_train, y_train, display_label=classes, normalize = normalize,
                         cmap = cmap, ax=axes[0])
    axes[0].set(title = 'Confusion Matrix Training')
    
    #plot ROC curve 
    plot_roc_curve(model, X_train, y_train, ax=axes[1])
    roc = axes[1]
    roc.legend()
    roc.plot([0,1], [0,1], ls=':')
    roc.grid()
    roc.set_title('Receiving Operator Characteristic Training')
    plt.show()
    
    #print testing classification report 
    header_ = label + " CLASSIFICATION REPORT TESTING "
    print(dashes, header_, dashes, sep='\n')
    print(classification_report(y_test, y_pred_test, target_names = classes))
    
    # display training figures 
    fig, axes = plt.subplots(figsize=(10,4), ncols=2)
    
    # confusion matrix 
    plot_confusion_matrix(model, X_test, y_test, display_label=classes, normalize = normalize,
                         cmap = cmap, ax=axes[0])
    axes[0].set(title = 'Confusion Matrix Testing')
    
    #plot ROC curve 
    plot_roc_curve(model, X_test, y_test, ax=axes[1])
    roc = axes[1]
    roc.legend()
    roc.plot([0,1], [0,1], ls=':')
    roc.grid()
    roc.set_title('Receiving Operator Characteristic Test')
    plt.show()

## Logistic Regression 

In [68]:
# creating a pipeline for LogisticRegression 
log_pipe = Pipeline([('preprocessing', preprocessing),
                    ('logreg', LogisticRegression(fit_intercept = False, solver='liblinear'))])

### log_pipe Vanilla Model 

In [70]:
# fit and transform 
log_pipe.fit(X_train, y_train)

# get prediction 
y_pred = log_pipe.predict(X_test)


In [ ]:
# classification report using function 
evaluate_classification(log_pipe,X_train, X_test, y_train, y_test, labels = 'Logistic Regression')

### log_pipe GridsearchCV

In [ ]:
# create params_grid for logistic regression 
params_grid = {}

# create grid search 
grid = GridSearchCV(log_pipe, params_grid, cv=3)

# fit x_train and y_train to grid 
grid.fit(X_train, y_train)

# observe combination of best params 
grid.best_params

In [ ]:
# evaluate model using function evaluate_classification and best_params(best_estimator)
evaluate_classification(grid.best_estimator_, X_train, X_test, y_train, y_test, labels ="Logistic Regression")

## KNN Model 

In [77]:
# creating a pipeline for LogisticRegression 
knn_pipe = Pipeline([('preprocessing', preprocessing),
                    ('knn', KNeighborsClassifier())])

### knn_pipe Vanilla Model 

In [ ]:
# fit and transform 
knn_pipe.fit(X_train, y_train)

# get prediction 
y_pred = knn_pipe.predict(X_test)


In [ ]:
# classification report using function 
evaluate_classification(knn_pipe,X_train, X_test, y_train, y_test, labels = 'KNN')

### knn_pipe GridsearchCV

In [ ]:
# create params_grid for KNeighborsClassifier  
params_grid = {}

# create grid search 
grid = GridSearchCV(knn_pipe, params_grid, cv=3)

# fit x_train and y_train to grid 
grid.fit(X_train, y_train)

# observe combination of best params 
grid.best_params

In [ ]:
# evaluate model using function evaluate_classification and best_params(best_estimator)
evaluate_classification(grid.best_estimator_, X_train, X_test, y_train, y_test, labels='KNN')

## Decision Tree

In [ ]:
# creating a pipeline for DecisionTreeClassifier 
tree_pipe = Pipeline([('preprocessing', preprocessing),
                    ('decision', DecisionTreeClassifier())])

### tree_pipe Vanilla Model 

In [ ]:
# fit and transform 
tree_pipe.fit(X_train, y_train)

# get prediction 
y_pred = tree_pipe.predict(X_test)

In [ ]:
# classification report using function 
evaluate_classification(tree_pipe,X_train, X_test, y_train, y_test, labels = 'Decision Tree')

### tree_pipe GridsearchCV

In [ ]:
# create params_grid for DecisionTreeClassifier  
params_grid = {}

# create grid search 
grid = GridSearchCV(tree_pipe, params_grid, cv=3)

# fit x_train and y_train to grid 
grid.fit(X_train, y_train)

# observe combination of best params 
grid.best_params

In [ ]:
# evaluate model using function evaluate_classification and best_params(best_estimator)
evaluate_classification(grid.best_estimator_, X_train, X_test, y_train, y_test, labels='Decision Tree')

## Bagged Trees 

In [81]:
# creating a pipeline for Bagging + DecisionTreeClassifier  
bag_pipe = Pipeline([('preprocessing', preprocessing),
                     ('Bagging', BaggingClassifier(DecisionTreeClassifier()))])

### bag_pipe Vanilla Model 

In [ ]:
# fit and transform 
bag_pipe.fit(X_train, y_train)

# get prediction 
y_pred = bag_pipe.predict(X_test)

In [ ]:
# classification report using function 
evaluate_classification(bag_pipe,X_train, X_test, y_train, y_test, labels = 'Bagged Trees')

### bag_pipe GridsearchCV

In [ ]:
# create params_grid for BaggingClassifier  
params_grid = {}

# create grid search 
grid = GridSearchCV(bag_pipe, params_grid, cv=3)

# fit x_train and y_train to grid 
grid.fit(X_train, y_train)

# observe combination of best params 
grid.best_params

In [ ]:
# evaluate model using function evaluate_classification and best_params(best_estimator)
evaluate_classification(grid.best_estimator_, X_train, X_test, y_train, y_test, labels='Bagged Trees')

## Random Forest 

In [ ]:
# creating a pipeline RandomForestClassifier  
forest_pipe = Pipeline([('preprocessing', preprocessing),
                     ('forest', RandomForestClassifier())])

### forest_pipe Vanilla Model 

In [ ]:
# fit and transform 
forest_pipe.fit(X_train, y_train)

# get prediction 
y_pred = forest_pipe.predict(X_test)

In [ ]:
# classification report using function 
evaluate_classification(bag_pipe,X_train, X_test, y_train, y_test, labels = 'Random Forest')

### forest_pipe GridsearchCV

In [ ]:
# create params_grid for RandomForestClassifier 
params_grid = {}

# create grid search 
grid = GridSearchCV(forest_pipe, params_grid, cv=3)

# fit x_train and y_train to grid 
grid.fit(X_train, y_train)

# observe combination of best params 
grid.best_params

In [ ]:
# evaluate model using function evaluate_classification and best_params(best_estimator)
evaluate_classification(grid.best_estimator_, X_train, X_test, y_train, y_test, labels='Random Forest')

# iNtrepretation

# Conclusion 

# Future Work 